In [6]:
import json
import glob
import os

def merge_json_files(input_pattern: str, output_filename: str):
    """
    주어진 패턴과 일치하는 모든 JSON 파일을 찾아 하나의 파일로 병합합니다.

    :param input_pattern: 찾을 파일의 패턴 (예: 'data/*.json')
    :param output_filename: 저장할 최종 파일의 이름
    """
    # 1. 병합할 JSON 파일 목록 찾기
    json_files = glob.glob(input_pattern)
    
    if not json_files:
        print(f"❌ 오류: '{input_pattern}' 패턴과 일치하는 파일을 찾을 수 없습니다.")
        return

    print(f"📄 총 {len(json_files)}개의 파일을 병합합니다: {json_files}")

    # 2. 모든 'questions'를 담을 빈 리스트 생성
    all_questions = []

    # 3. 각 파일을 순회하며 'questions' 리스트를 읽고 합치기
    for file_path in json_files:
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                # 'questions' 키가 존재하고, 그 값이 리스트인 경우에만 추가
                if 'questions' in data and isinstance(data['questions'], list):
                    all_questions.extend(data['questions'])
                else:
                    print(f"⚠️ 경고: '{file_path}' 파일에 'questions' 리스트가 없거나 형식이 잘못되어 건너뜁니다.")
        except json.JSONDecodeError:
            print(f"⚠️ 경고: '{file_path}' 파일이 올바른 JSON 형식이 아니므로 건너뜁니다.")
        except Exception as e:
            print(f"'{file_path}' 처리 중 오류 발생: {e}")

    # 4. 최종적으로 합쳐진 데이터 구조 만들기
    final_data = {"questions": all_questions}

    # 5. 새로운 JSON 파일에 저장
    with open(output_filename, 'w', encoding='utf-8') as f:
        # indent=4: 보기 좋게 4칸 들여쓰기 적용
        # ensure_ascii=False: 한글 등 비-ASCII 문자가 깨지지 않도록 설정
        json.dump(final_data, f, indent=4, ensure_ascii=False)

    print(f"\n✅ 병합이 완료되었습니다. 총 {len(all_questions)}개의 질문이 '{output_filename}' 파일에 저장되었습니다.")

# --- 메인 실행 부분 ---
if __name__ == "__main__":
    # ⭐️ 여기에 병합할 JSON 파일들이 있는 폴더 경로와 파일 패턴을 지정하세요.
    INPUT_PATTERN = '/workspace/CyberMetric/*.json' 
    
    # ★★★ 수정된 부분: 저장 경로를 /workspace/로 명시 ★★★
    OUTPUT_FILE = '/workspace/merged_questions.json'
    
    merge_json_files(INPUT_PATTERN, OUTPUT_FILE)

📄 총 4개의 파일을 병합합니다: ['/workspace/CyberMetric/CyberMetric-10000-v1.json', '/workspace/CyberMetric/CyberMetric-2000-v1.json', '/workspace/CyberMetric/CyberMetric-500-v1.json', '/workspace/CyberMetric/CyberMetric-80-v1.json']

✅ 병합이 완료되었습니다. 총 12760개의 질문이 '/workspace/merged_questions.json' 파일에 저장되었습니다.


In [7]:
import json 

file_path = "/workspace/merged_questions.json"
with open(file_path, "r", encoding="utf-8") as f:
    dataset = json.load(f)

print(len(dataset["questions"]))

12760


In [14]:
document_len = len(dataset["questions"])
real_data = dataset["questions"]

all_docs = []

for i in range(document_len):

    question = real_data[i]["question"]
    answer = real_data[i]["answers"].get(real_data[i]["solution"])

    docs = {
        "question" : question,
        "answer" : answer
    }

    all_docs.append(docs)

print(all_docs[10])

{'question': 'Which of the following authentication factors involves something you know?', 'answer': 'Username'}


In [15]:
%pip install transformers torch sentencepiece


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 9.5 MB/s eta 0:00:00ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import torch
import json
from transformers import pipeline
from tqdm.auto import tqdm


MODEL_NAME = "facebook/nllb-200-distilled-600M"
OUTPUT_FILENAME = "/workspace/translated_qa_ko.jsonl"

print(f"'{MODEL_NAME}' 번역 모델을 로딩합니다... (시간이 걸릴 수 있습니다)")

# GPU 사용 가능 여부 확인 후 장치 설정
device = 0 if torch.cuda.is_available() else -1

# 번역을 위한 pipeline 객체 생성
translator = pipeline(
    'translation',
    model=MODEL_NAME,
    tokenizer=MODEL_NAME,
    src_lang='eng_Latn',  # 소스 언어: 영어 (Latin script)
    tgt_lang='kor_Hang',  # 타겟 언어: 한국어 (Hangul script)
    device=device         # GPU가 있으면 사용, 없으면 CPU(-1) 사용
)

print("✅ 번역 모델 로딩 완료.")

# --- 3. 데이터 번역 실행 ---

# 번역된 결과를 저장할 새로운 리스트
translated_docs = []

print(f"\n총 {len(all_docs)}개의 항목에 대한 번역을 시작합니다...")

# tqdm을 사용하여 진행 상황을 시각적으로 표시
for doc in tqdm(all_docs, desc="번역 진행 중"):
    # 질문과 답변 텍스트를 리스트로 묶어 한 번에 번역 (더 효율적)
    texts_to_translate = [
        doc.get("question", ""),
        doc.get("answer", "")
    ]
    
    # 번역 실행
    translation_results = translator(texts_to_translate, max_length=512)
    
    # 번역 결과 추출
    translated_question = translation_results[0]['translation_text']
    translated_answer = translation_results[1]['translation_text']
    
    # 번역된 내용으로 새로운 딕셔너리 생성
    translated_doc = {
        "question": translated_question,
        "answer": translated_answer
    }
    
    # 최종 리스트에 추가
    translated_docs.append(translated_doc)

print("\n✅ 번역이 완료되었습니다.")

# --- 4. 번역 결과 확인 및 파일 저장 ---

print("\n--- [번역 결과 샘플] ---")
print(json.dumps(translated_docs[0], indent=2, ensure_ascii=False))

# 번역된 전체 결과를 jsonl 파일로 저장
with open(OUTPUT_FILENAME, 'w', encoding='utf-8') as f:
    for doc in translated_docs:
        # 각 딕셔너리를 JSON 문자열로 변환하여 파일에 한 줄씩 씀
        f.write(json.dumps(doc, ensure_ascii=False) + '\n')

print(f"\n💾 모든 번역 결과가 '{OUTPUT_FILENAME}' 파일에 저장되었습니다.")

'facebook/nllb-200-distilled-600M' 번역 모델을 로딩합니다... (시간이 걸릴 수 있습니다)


config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

...

Device set to use cuda:0


✅ 번역 모델 로딩 완료.

총 12760개의 항목에 대한 번역을 시작합니다...


번역 진행 중:   0%|          | 0/12760 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



✅ 번역이 완료되었습니다.

--- [번역 결과 샘플] ---
{
  "question": "다음 중 어느 것 은 정보 의 비밀 에 관한 것 입니까?",
  "answer": "비공개성"
}

💾 모든 번역 결과가 'translated_qa_ko.jsonl' 파일에 저장되었습니다.


In [ ]:
# 번역된 전체 결과를 jsonl 파일로 저장
OUTPUT_FILENAME = "/workspace/translated_qa_ko.jsonl"


with open(OUTPUT_FILENAME, 'w', encoding='utf-8') as f:
    for doc in translated_docs:
        # 각 딕셔너리를 JSON 문자열로 변환하여 파일에 한 줄씩 씀
        f.write(json.dumps(doc, ensure_ascii=False) + '\n')

print(f"\n💾 모든 번역 결과가 '{OUTPUT_FILENAME}' 파일에 저장되었습니다.")


💾 모든 번역 결과가 '/workspace/translated_qa_ko.jsonl' 파일에 저장되었습니다.


: 